In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
from tensorflow.keras.layers import Dropout
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Overall Notes/Ideas

* Consider comparing lithium against trace elements and lithium againt major elements
  * In other words, lithium against 'ppm' elements and lithium against 'WT%' elements.

####################################################


The initial dataset is taken from the GEOROC geochemical database and is a large dataset of micas taken from different papers.

* Column R gives the kind of mica (mineral name) – you should **exclude** celadonite, glauconite, hydromica, hydromuscovite, margarite, sericite and yangzhumingite in the first instance.

* That leaves two broad kinds of mica – muscovite (which is essentially free of Fe and Mg) and biotite (which contains Fe and Mg).

I’m sure you’ll have lots of questions about all this! So just get in touch when you want to.



It's also worth saying that I’m sure this is also **not complete**. For example, I know there was a very recent paper **(Breiter et al. 2023)** which has a lot of data presented in figures but not in dataset form – there are probably also others. A detailed literature search might produce more, and there may be good ways to extract data from digital figures.

In [ ]:
ini = pd.read_excel('FULL_GEOROC_2022_12_SGFTFN_MICA.xlsx')

## Further Instructions

You can delete age information, that’s no problem.
Age represents a geological age (i.e. timing of when the mineral formed) so is not really relevant.

Similarly, you’ll find columns with **isotopic compositions (e.g. AR36(CCMSTP/G), AR39(MOL/G))** which are also not relevant to what you’re doing. Those can be ignored, hidden or deleted.

## Drop Non-Relevant Columns

Drop He3 (CR), He4 (CS), Ne20 (CY), the Ar36-Ar40 columns (DG-DO), K40 (DQ), Ca42 (DS), Ca43 (DT), Ge73 (EG), Ge74 (EH), KR84 (EL), Mo95-100 (ES-EW), XE132 (FC), RA226 (GC) and everything from GG to JC inclusive.

In [ ]:
# We can update this as we go
non_relevant = ['AGE(KA)', 'AGE(MA)', 'CITATION', 'AR36(CCMSTP/G)',
'AR39(CCMSTP/G)', 'AR39(MOL/G)', 'AR40(AT/G)',
'AR40(CCMSTP/G)', 'AR40(MOL/G)', 'AR40(PPM)', 'AR40(PPB)',
'HE3(CCMSTP/G)', 'HE4(CCMSTP/G)', 'K40(PPM)',
'NE20(CCMSTP/G)', 'CA42(PPM)', 'CA43(PPM)', 'GE73(PPM)',
'GE74(PPM)', 'KR84(CCMSTP/G)', 'MO95(PPM)', 'MO97(PPM)',
'MO98(PPM)', 'MO100(PPM)', 'XE132(CCMSTP/G)', 'RA226_ACT(BQ/KG)',
'ND143_ND144','EPSILON_ND','EPSILON_ND_INI','SM147_ND144',
'SR87_SR86', 'RB87_SR86', 'PB206_PB204','PB207_PB204',
'PB207_PB206', 'PB208_PB204', 'PB208_PB206','HF176_HF177', 'LU176_HF177',
'HE4_HE3', 'K40_AR36','K40_CA44', 'AR36_AR38', 'AR36_AR39',
'AR36_AR40','AR37_AR39', 'AR37_AR40','AR38_AR36', 'AR38_AR39', 'AR38_AR40',
'AR39_AR36','AR39_AR40', 'AR40_AR36', 'AR40_AR38', 'AR40_AR39', 'AR40_K40', 'PB207_U235',
'PB210_TH230_ACT', 'RA226_TH230_ACT', 'TH230_TH232_ACT', 'TH230_U238_ACT', 'TH232_PB204',
'TH232_PB208', 'TH232_U238', 'U234_U238_ACT','U235_PB204', 'U238_PB204', 'U238_PB206',
'U238_TH230_ACT', 'U238_TH232_ACT', 'N2(MOL/G)', 'D7LI(VS LSVEC)', 'D7LI(VS NBS8545)',
'D7LI(VS IRMM-016)', 'D11B(VS NBS951)', 'B11_B10', 'D13C(VS VPDB)', 'D15N(PER MIL)',
'D18O(PER MIL)', 'D18O(VS SMOW)', 'D18O(VS VSMOW)', 'D25MG(VS DSM3)', 'D26MG(VS DSM3)',
'CA40_CA44', 'D41K(VS SRM3141A)', 'D44_40CA(VS SRM915A)', 'D44_42CA(VS SRM915A)',
'D49TI(VS OL-TI)', 'D56FE(VS IRMM-014)', 'D57FE(VS IRMM-014)', 'D60NI(VS NBS986)',
'D66ZN(VS JMC3-0749)', 'D68ZN(VS JMC3-0749)', 'D98_95MO(PER MIL)', 'D98MO(VS NIST3134)',
'D98_95MO(VS NIST3134)', 'D137_134BA(VS SRM3104A)', 'D138_134BA(VS SRM3104A)',
'DD(PER MIL)', 'DD(VS SMOW)', 'DD(VS VSMOW)', 'SM2O3(WT%)', 'EU2O3(WT%)',
'GD2O3(WT%)','DY2O3(WT%)','SO2(WT%)','SO3(WT%)','S(WT%)','H2S(WT%)','LOI(WT%)',
'O(WT%)', 'YB2O3(WT%)', 'PR2O3(WT%)', 'AS2O3(WT%)', 'CH4(WT%)','O2(WT%)',
'CO(WT%)', 'H2O(WT%)', 'H2OP(WT%)', 'H2OM(WT%)','CO2(WT%)', 'OH(WT%)',
'HE3(CCMSTP/G)', 'HE4(CCMSTP/G)', 'NE20(CCMSTP/G)', 'AR36(CCMSTP/G)',
'AR39(CCMSTP/G)', 'AR39(MOL/G)', 'AR40(AT/G)', 'AR40(CCMSTP/G)', 'AR40(MOL/G)',
'AR40(PPM)', 'AR40(PPB)', 'K40(PPM)', 'CA42(PPM)', 'CA43(PPM)', 'GE73(PPM)',
'GE74(PPM)', 'KR84(CCMSTP/G)', 'MO95(PPM)', 'MO97(PPM)', 'MO98(PPM)',
'MO100(PPM)', 'XE132(CCMSTP/G)', 'ALBITE(MOL%)', 'ANORTHITE(MOL%)', 'ANNITE(MOL%)',
'ENSTATITE(MOL%)', 'FERROSILITE(MOL%)', 'MARGARITE(MOL%)','MUSCOVITE(MOL%)',
'ORTHOCLASE(MOL%)', 'PARAGONITE(MOL%)','PHLOGOPITE(MOL%)', 'WOLLASTONITE(MOL%)',
'N(PPM)', 'S(PPM)', 'BR(PPM)', 'GRAIN SIZE', 'SAMPLE NAME']

# You can scrap AJ-AN inclusive, AU, BR, CB-CI inclusive and CP.
# Other elements that won’t be of interest for your regression include BS-BV, CA

In [ ]:
# Drop Non-Relevant Columns
df = ini.drop(non_relevant, axis = 1)
df.columns

### **Exclude** CELADONITE, GLAUCONITE, HYDROMICA, HYDROMUSCOVITE, MARGARITE, SERICITE and YANGZHUMINGITE from the column titled 'MINERAL'

In [ ]:
df['MINERAL'].value_counts()

In [ ]:
excluded = ['CELADONITE', 'GLAUCONITE', 'HYDROMICA', 'HYDROMUSCOVITE',
            'MARGARITE', 'SERICITE', 'YANGZHUMINGITE']

In [ ]:
extract = ~df['MINERAL'].isin(excluded)
df = df[extract]

In [ ]:
df['MINERAL'].value_counts()

# Mol Calculations

In [ ]:
# We will write mol, wt%, and ppm calculation functions here

def wt2ppm(item): # to ppm
  return item * 10000

def ppm2wt(item): # to WT%
  return item / 10000

def ppb2m(item): # to ppm
  return item / 1000

def ppb2wt(item): # to WT%
  return item / 10000000


In [ ]:
# Functions to convert materials from their oxide form to elements
def ZRO2(item):
  return item*(91.2/(91.2 + 32))

def HFO2(item):
  return item*(178.5/(178.5 + 32))

def THO2(item):
  return item*(232/(232 + 32))

def UO2(item):
  return item*(238/(238 + 32))

def UO3(item):
  return item*(238/(238 + 48))

def CR2O3(item):
  return item*((2*52)/(2*52 + 48))

def LA2O3(item):
  return item*((2*138.9)/(2*138.9 + 48))

def CE2O3(item):
  return item*((2*140.1)/(2*140.1 + 48))

def ND2O3(item):
  return item*((2*144.2)/(2*144.2 + 48))

def Y2O3(item):
  return item*((2*88.9)/(2*88.9 + 48))

def V2O3(item):
  return item*((2*50.9)/(2*50.9 + 48))

def V2O5(item):
  return item*((2*50.9)/(2*50.9 + 80))

def NB2O3(item):
  return item*((2*92.9)/(2*92.9 + 48))

def NB2O5(item):
  return item*((2*92.9)/(2*92.9 + 80))

def TA2O5(item):
  return item*((2*180.9)/(2*180.9 + 80))

def WO3(item):
  return item*((183.8)/(183.8 + 48))

def BAO(item):
  return item*(137.3/(137.3 + 16))

def SRO(item):
  return item*(87.6/(87.6 + 16))

def PBO(item):
  return item*(207.2/(207.2 + 16))

def SNO2(item):
  return item*(118.7/(118.7 + 32))

def NIO(item):
  return item*(58.7/(58.7 + 16))

def ZNO(item):
  return item*(65.4/(65.4 + 16))

def COO(item):
  return item*(58.9/(58.9 + 16))

def CUO(item):
  return item*(63.5/(63.5 + 16))

def CS2O(item):
  return item*((2*132.9)/(2*132.9 + 16))

def RB2O(item):
  return item*((2*85.5)/(2*85.5 + 16))

def LI2O(item):
  return item*((2*6.9)/(2*6.9 + 16))

def F2O(item):
  return item*((2*19)/(2*19 + 16))

def CL2O(item): # BZ
  return item*((2*35.5)/(2*35.5 + 16))


##############################################################################

# You can scrap AJ-AN inclusive, AU, BR, CA-CI inclusive and CP.
# Other elements that won’t be of interest for your regression include BS-BV

In [ ]:
# Functions to convert certain elements to their oxide form
# Elements that should be left as oxide include SiO2, TiO2,
# Al2O3, FeO (or Fe2O3), MnO, MgO, CaO, Na2O, K2O, P2O5, H2O (if present).

# SiO2
def Si(revert):
  return revert*((28.1 + 32)/28.1)

# Al2O3
def Al(revert):
  return revert*((2*27 + 48)/(2*27))

# Fe2O3 to FeO
def FeO(revert):
  return revert*((2*(55.8 + 16))/(2*55.8 + 48))

# Fe to FeO
def Fe(revert):
  return revert*((55.8 + 16)/(58.8))

# MgO
def Mg(revert):
  return revert*((24.3 + 16)/24.3)

# Na2O
def Na(revert):
  return revert*((2*23 + 16)/(2*23))

# K2O
def K(revert):
  return revert*((2*39.1 + 16)/(2*39.1))

# P2O5
def P(revert):
  return revert*((2*31 + 80)/(2*31))

# CaO
def Ca(revert):
  return revert*((40.1 + 16)/40.1)

# MnO
def Mn(revert):
  return revert*((54.9 + 16)/54.9)

In [ ]:
def convert_mixed_value(value):
    if isinstance(value, (float, int)):
        return float(value)
    if "\\" in value:
        parts = value.split('\\')
        number = parts[0].strip()
        return float(number)
    else:
        return float(value)

## Conversions and Merging

#### Notes

* FeOT and FeO should be merged in advance
* Same with Fe2O3T and Fe2O3

## Pre-Merging

In [ ]:
# Merge FeOT and FeO
df['FEO(WT%)'] = df['FEOT(WT%)'].fillna(df['FEO(WT%)'])
# Merge Fe2O3T and Fe2O3
df['FE2O3(WT%)'] = df['FE2O3T(WT%)'].fillna(df['FE2O3(WT%)'])

## Transforming Columns to Floats

Some columns contain non float and non int type data. This makes them incompatible with our above functions.

We will iterate over these values to reduce code duplication and ensure conversion consistency.



In [ ]:
_numericals = ['SIO2(WT%)', 'TIO2(WT%)', 'AL2O3(WT%)', 'FEO(WT%)', 'CAO(WT%)',
    'MGO(WT%)', 'MNO(WT%)', 'K2O(WT%)', 'NA2O(WT%)', 'P2O5(WT%)', 'LI(PPM)',
    'BE(PPM)', 'B(PPM)', 'F(PPM)', 'CL(PPM)', 'SC(PPM)',
    'TI(PPM)', 'V(PPM)', 'CR(PPM)', 'CO(PPM)', 'NI(PPM)', 'CU(PPM)',
    'ZN(PPM)', 'GA(PPM)', 'GE(PPM)', 'AS(PPM)', 'SE(PPM)',
    'RB(PPM)', 'SR(PPM)', 'Y(PPM)', 'ZR(PPM)', 'NB(PPM)', 'MO(PPM)',
    'AG(PPM)', 'CD(PPM)', 'IN(PPM)', 'SN(PPM)', 'SB(PPM)', 'TE(PPM)',
    'CS(PPM)', 'BA(PPM)', 'LA(PPM)', 'CE(PPM)', 'PR(PPM)', 'ND(PPM)',
    'SM(PPM)', 'EU(PPM)', 'GD(PPM)', 'TB(PPM)', 'DY(PPM)', 'HO(PPM)',
    'ER(PPM)', 'TM(PPM)', 'YB(PPM)', 'LU(PPM)', 'HF(PPM)', 'TA(PPM)',
    'W(PPM)', 'AU(PPM)', 'HG(PPM)', 'TL(PPM)', 'PB(PPM)', 'BI(PPM)',
    'TH(PPM)', 'U(PPM)', 'CR2O3(WT%)', 'BAO(WT%)', 'NIO(WT%)',
            'LI2O(WT%)', 'RB2O(WT%)', 'K(WT%)']
for col in _numericals:
  df[col] = df[col].apply(convert_mixed_value)

## Column Conversions

### Oxides

In [ ]:
# Here, we convert each relevant column and will merge them in later code cells

# We use .pipe() to chain functions
# Converting WT% to WT% Oxides
# SiO2
df['SI(WT%)'] = df['SI(WT%)'].apply(Si)
df['SI(PPM)'] = df['SI(PPM)'].pipe(ppm2wt).pipe(Si)

# Al2O3
df['AL(WT%)'] = df['AL(WT%)'].apply(Al)
df['AL(PPM)'] = df['AL(PPM)'].pipe(ppm2wt).pipe(Al)

# Fe2O3 (FeOT and Fe2O3T are the only ones measured with the T method)
df['FE2O3(WT%)'] = df['FE2O3(WT%)'].apply(FeO)
# Fe
df['FE(WT%)'] = df['FE(WT%)'].apply(Fe)
df['FE(PPM)'] = df['FE(PPM)'].pipe(ppm2wt).pipe(Fe)

# MnO
df['MN(PPM)'] = df['MN(PPM)'].pipe(ppm2wt).pipe(Mn)

# MgO
df['MG(WT%)'] = df['MG(WT%)'].apply(Mg)
df['MG(PPM)'] = df['MG(PPM)'].pipe(ppm2wt).pipe(Mg)

# CaO
df['CA(PPM)'] = df['CA(PPM)'].pipe(ppm2wt).pipe(Ca)

# Na2O
df['NA(WT%)'] = df['NA(WT%)'].apply(Na)
df['NA(PPM)'] = df['NA(PPM)'].pipe(ppm2wt).pipe(Na)

# K2O
df['K(WT%)'] = df['K(WT%)'].apply(K)
df['K(PPM)'] = df['K(PPM)'].pipe(ppm2wt).pipe(K)

# P2O5
df['P(PPM)'] = df['P(PPM)'].pipe(ppm2wt).pipe(P)

### Non-Oxides

In [ ]:
# Trace elements; those in lower concentrations, should be left as ppm
# MAJOR ELEMENTS - SiO2, TiO2, Al2O3, FeO, MnO, MgO, CaO, Na2O, K2O, P2O5

# ZrO2 // ZR(PPM) - EP
df['ZRO2(WT%)'] = df['ZRO2(WT%)'].pipe(ZRO2).pipe(wt2ppm)

# HFO2 // HF(PPM) - FT
df['HFO2(WT%)'] = df['HFO2(WT%)'].pipe(HFO2).pipe(wt2ppm)

# THO2 // TH(PPM) - GD
df['THO2(WT%)'] = df['THO2(WT%)'].pipe(THO2).pipe(wt2ppm)

# UO2, UO3 // U(PPM), U(PPB) - GE, GF
df['UO2(WT%)'] = df['UO2(WT%)'].pipe(UO2).pipe(wt2ppm)
df['UO3(WT%)'] = df['UO3(WT%)'].pipe(UO3).pipe(wt2ppm)
df['U(PPB)'] = df['U(PPB)'].pipe(ppb2m)

# CR2O3 // CR(PPM) - DX
df['CR2O3(WT%)'] = df['CR2O3(WT%)'].pipe(CR2O3).pipe(wt2ppm)
df['CR(WT%)'] = df['CR(WT%)'].pipe(wt2ppm)

# LA2O3 // LA(PPM) - FF
df['LA2O3(WT%)'] = df['LA2O3(WT%)'].pipe(LA2O3).pipe(wt2ppm)

# CE2O3 // CE(PPM) - FG
df['CE2O3(WT%)'] = df['CE2O3(WT%)'].pipe(CE2O3).pipe(wt2ppm)

# ND2O3 // ND(PPM) - FI
df['ND2O3(WT%)'] = df['ND2O3(WT%)'].pipe(ND2O3).pipe(wt2ppm)

# Y2O3 // Y(PPM) - EO
df['Y2O3(WT%)'] = df['Y2O3(WT%)'].pipe(Y2O3).pipe(wt2ppm)

# V2O3, V2O5 // V(PPM) - DW
df['V2O3(WT%)'] = df['V2O3(WT%)'].pipe(V2O3).pipe(wt2ppm)
df['V2O5(WT%)'] = df['V2O3(WT%)'].pipe(V2O5).pipe(wt2ppm)

# NB2O3, NB2O5 // NB(PPM) - EQ
df['NB2O3(WT%)'] = df['NB2O3(WT%)'].pipe(NB2O3).pipe(wt2ppm)
df['NB2O5(WT%)'] = df['NB2O3(WT%)'].pipe(NB2O5).pipe(wt2ppm)

# TA2O5 // TA(PPM) - FU
df['TA2O5(WT%)'] = df['TA2O5(WT%)'].pipe(TA2O5).pipe(wt2ppm)

# WO3 // W(PPM) - FV
df['WO3(WT%)'] = df['WO3(WT%)'].pipe(WO3).pipe(wt2ppm)

# BAO // BA(PPM) - FE
df['BAO(WT%)'] = df['BAO(WT%)'].pipe(BAO).pipe(wt2ppm)

# SRO // SR(PPM) - EN
df['SRO(WT%)'] = df['SRO(WT%)'].pipe(SRO).pipe(wt2ppm)

# PBO // PB(PPM) - FZ
df['PBO(WT%)'] = df['PBO(WT%)'].pipe(PBO).pipe(wt2ppm)
df['PB(PPB)'] = df['PB(PPB)'].pipe(ppb2m)

# SNO2 // SN(PPM) - EZ
df['SNO2(WT%)'] = df['SNO2(WT%)'].pipe(SNO2).pipe(wt2ppm)

# NIO // NI(PPM) - EB
df['NIO(WT%)'] = df['NIO(WT%)'].pipe(NIO).pipe(wt2ppm)

# ZNO // ZN(PPM) - ED
df['ZNO(WT%)'] = df['ZNO(WT%)'].pipe(ZNO).pipe(wt2ppm)

# COO // CO(PPM) - EA
df['COO(WT%)'] = df['COO(WT%)'].pipe(COO).pipe(wt2ppm)

# CUO // CU(PPM) - EC
df['CUO(WT%)'] = df['CUO(WT%)'].pipe(CUO).pipe(wt2ppm)

# CS2O // CS(PPM) - FD
df['CS2O(WT%)'] = df['CS2O(WT%)'].pipe(CS2O).pipe(wt2ppm)

# RB2O // RB(PPM) - EM
df['RB2O(WT%)'] = df['RB2O(WT%)'].pipe(RB2O).pipe(wt2ppm)

# LI2O // LI(PPM) - CT
df['LI2O(WT%)'] = df['LI2O(WT%)'].pipe(LI2O).pipe(wt2ppm)

# F2O // F(PPM) - CX
df['F(WT%)'] = df['F(WT%)'].pipe(wt2ppm)
df['F2O(WT%)'] = df['F2O(WT%)'].pipe(F2O).pipe(wt2ppm)

# CL2O // CL(PPM) - DF
df['CL(WT%)'] = df['CL(WT%)'].pipe(wt2ppm)
df['CL2O(WT%)'] = df['CL2O(WT%)'].pipe(CL2O).pipe(wt2ppm)

## Column Merging

IMPORTANT - When merging the columns, data values need to stay **inplace**.
This is because each value corresponds to measurements taken from specific rocks/minerals.

Thus, we will use .fillna() with the 'inplace' argument to merge these columns.



### Oxides

In [ ]:
# Oxides (We will leave major elements in WT% form)

# SIO2
df['SIO2(WT%)'] = df['SIO2(WT%)'].fillna(df['SI(PPM)']).fillna(df['SI(WT%)'])

# AL2O3
df['AL2O3(WT%)'] = df['AL2O3(WT%)'].fillna(df['AL(WT%)']).fillna(df['AL(PPM)'])

# FEO
df['FEO(WT%)'] = df['FEO(WT%)'].fillna(df['FE2O3(WT%)']).fillna(df['FE(WT%)']).fillna(df['FE(PPM)'])

# MNO
df['MNO(WT%)'] = df['MNO(WT%)'].fillna(df['MN(PPM)'])

# MGO
df['MGO(WT%)'] = df['MGO(WT%)'].fillna(df['MG(PPM)']).fillna(df['MG(WT%)'])

# CAO
df['CAO(WT%)'] = df['CAO(WT%)'].fillna(df['CA(PPM)'])

# NA2O
df['NA2O(WT%)'] = df['NA2O(WT%)'].fillna(df['NA(WT%)']).fillna(df['NA(PPM)'])

# K20
df['K2O(WT%)'] = df['K2O(WT%)'].fillna(df['K(WT%)']).fillna(df['K(PPM)'])

# P2O5
df['P2O5(WT%)'] = df['P2O5(WT%)'].fillna(df['P(PPM)'])

### Non-Oxides

In [ ]:
# Trace elements should be left in ppm format

# ZR
df['ZR(PPM)'] = df['ZR(PPM)'].fillna(df['ZRO2(WT%)'])

# HF
df['HF(PPM)'] = df['HF(PPM)'].fillna(df['HFO2(WT%)'])

# TH
df['TH(PPM)'] = df['TH(PPM)'].fillna(df['THO2(WT%)'])

# U
df['U(PPM)'] = df['U(PPM)'].fillna(df['UO2(WT%)']).fillna(df['UO3(WT%)']).fillna(df['U(PPB)'])

# CR
df['CR(PPM)'] = df['CR(PPM)'].fillna(df['CR2O3(WT%)']).fillna(df['CR(WT%)'])

# LA
df['LA(PPM)'] = df['LA(PPM)'].fillna(df['LA2O3(WT%)'])

# CE
df['CE(PPM)'] = df['CE(PPM)'].fillna(df['CE2O3(WT%)'])

# ND
df['ND(PPM)'] = df['ND(PPM)'].fillna(df['ND2O3(WT%)'])

# Y
df['Y(PPM)'] = df['Y(PPM)'].fillna(df['Y2O3(WT%)'])

# V
df['V(PPM)'] = df['V(PPM)'].fillna(df['V2O3(WT%)']).fillna(df['V2O5(WT%)'])

# NB
df['NB(PPM)'] = df['NB(PPM)'].fillna(df['NB2O3(WT%)']).fillna(df['NB2O5(WT%)'])

# TA
df['TA(PPM)'] = df['TA(PPM)'].fillna(df['TA2O5(WT%)'])

# W
df['W(PPM)'] = df['W(PPM)'].fillna(df['WO3(WT%)'])

# BA
df['BA(PPM)'] = df['BA(PPM)'].fillna(df['BAO(WT%)'])

# SR
df['SR(PPM)'] = df['SR(PPM)'].fillna(df['SRO(WT%)'])

# PB
df['PB(PPM)'] = df['PB(PPM)'].fillna(df['PBO(WT%)']).fillna(df['PB(PPB)'])

# SN
df['SN(PPM)'] = df['SN(PPM)'].fillna(df['SNO2(WT%)'])

# NI
df['NI(PPM)'] = df['NI(PPM)'].fillna(df['NIO(WT%)'])

# ZN
df['ZN(PPM)'] = df['ZN(PPM)'].fillna(df['ZNO(WT%)'])

# CO
df['CO(PPM)'] = df['CO(PPM)'].fillna(df['COO(WT%)'])

# CU
df['CU(PPM)'] = df['CU(PPM)'].fillna(df['CUO(WT%)'])

# CS
df['CS(PPM)'] = df['CS(PPM)'].fillna(df['CS2O(WT%)'])

# RB
df['RB(PPM)'] = df['RB(PPM)'].fillna(df['RB2O(WT%)'])

# LI
df['LI(PPM)'] = df['LI(PPM)'].fillna(df['LI2O(WT%)'])

# F
df['F(PPM)'] = df['F(PPM)'].fillna(df['F(WT%)']).fillna(df['F2O(WT%)'])

# CL
df['CL(PPM)'] = df['CL(PPM)'].fillna(df['CL(WT%)']).fillna(df['CL2O(WT%)'])


### Drop columns that have been merged

In [ ]:
postmerged = ['SI(WT%)', 'SI(PPM)', 'AL(WT%)', 'AL(PPM)', 'FE2O3(WT%)',
              'FE2O3T(WT%)', 'FEOT(WT%)', 'MN(PPM)', 'MG(WT%)', 'MG(PPM)',
              'CA(PPM)', 'NA(WT%)', 'NA(PPM)', 'K(WT%)', 'K(PPM)',
              'P(PPM)', 'ZRO2(WT%)', 'HFO2(WT%)', 'THO2(WT%)', 'UO2(WT%)',
              'UO3(WT%)', 'U(PPB)', 'CR2O3(WT%)', 'CR(WT%)', 'LA2O3(WT%)',
              'CE2O3(WT%)', 'ND2O3(WT%)', 'Y2O3(WT%)', 'V2O3(WT%)', 'V2O5(WT%)',
              'NB2O3(WT%)', 'NB2O5(WT%)', 'TA2O5(WT%)', 'WO3(WT%)', 'BAO(WT%)',
              'SRO(WT%)', 'PBO(WT%)', 'PB(PPB)','SNO2(WT%)', 'NIO(WT%)',
              'ZNO(WT%)', 'COO(WT%)', 'CUO(WT%)', 'CS2O(WT%)', 'RB2O(WT%)',
              'LI2O(WT%)', 'F(WT%)', 'F2O(WT%)', 'CL(WT%)', 'CL2O(WT%)',
              'FE(WT%)', 'FE(PPM)', 'AR(MOL/G)']

In [ ]:
df = df.drop(postmerged, axis = 1)
df.columns

We can now drop all rows containing NaNs from the LI(PPM) column as we have successfully merged them.

In [ ]:
df = df.dropna(subset=['LI(PPM)'])
df.reset_index(drop = True, inplace = True)
df

####################################################################
# Modelling


In [ ]:
numericals = ['SIO2(WT%)', 'TIO2(WT%)', 'AL2O3(WT%)', 'FEO(WT%)', 'CAO(WT%)',
       'MGO(WT%)', 'MNO(WT%)', 'K2O(WT%)', 'NA2O(WT%)', 'P2O5(WT%)', 'LI(PPM)',
       'BE(PPM)', 'B(PPM)', 'F(PPM)', 'CL(PPM)', 'SC(PPM)',
       'TI(PPM)', 'V(PPM)', 'CR(PPM)', 'CO(PPM)', 'NI(PPM)', 'CU(PPM)',
       'ZN(PPM)', 'GA(PPM)', 'GE(PPM)', 'AS(PPM)', 'SE(PPM)',
       'RB(PPM)', 'SR(PPM)', 'Y(PPM)', 'ZR(PPM)', 'NB(PPM)', 'MO(PPM)',
       'AG(PPM)', 'CD(PPM)', 'IN(PPM)', 'SN(PPM)', 'SB(PPM)', 'TE(PPM)',
       'CS(PPM)', 'BA(PPM)', 'LA(PPM)', 'CE(PPM)', 'PR(PPM)', 'ND(PPM)',
       'SM(PPM)', 'EU(PPM)', 'GD(PPM)', 'TB(PPM)', 'DY(PPM)', 'HO(PPM)',
       'ER(PPM)', 'TM(PPM)', 'YB(PPM)', 'LU(PPM)', 'HF(PPM)', 'TA(PPM)',
       'W(PPM)', 'AU(PPM)', 'HG(PPM)', 'TL(PPM)', 'PB(PPM)', 'BI(PPM)',
       'TH(PPM)', 'U(PPM)', 'LATITUDE (MIN.)', 'LATITUDE (MAX.)',
       'LONGITUDE (MIN.)', 'LONGITUDE (MAX.)']

#'ELEVATION (MIN.)','ELEVATION (MAX.)', 'DRILLING DEPTH (MIN.)',
# 'DRILLING DEPTH (MAX.)'

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.impute import SimpleImputer, KNNImputer

# # Store results
# results = {}

# # Normalize the data
# scaler = MinMaxScaler()
# X_normalized = scaler.fit_transform(df[numericals])
# X_normalized_df = pd.DataFrame(X_normalized, columns=numericals)

# # Split data for evaluation
# X_train, X_test, y_train, y_test = train_test_split(X_normalized_df, df['LI(PPM)'], test_size=0.3, random_state=42)

# # Mean/Median Imputation
# for strategy in ['mean', 'median']:
#     imputer = SimpleImputer(strategy=strategy)
#     X_train_imputed = imputer.fit_transform(X_train)
#     X_test_imputed = imputer.transform(X_test)

#     model = RandomForestRegressor()
#     model.fit(X_train_imputed, y_train)
#     predictions = model.predict(X_test_imputed)
#     mae = mean_absolute_error(y_test, predictions)

#     results[f'SimpleImputer ({strategy})'] = mae

# # KNN Imputation
# knn_imputer = KNNImputer(n_neighbors=2)
# X_train_imputed = knn_imputer.fit_transform(X_train)
# X_test_imputed = knn_imputer.transform(X_test)

# model = RandomForestRegressor()
# model.fit(X_train_imputed, y_train)
# predictions = model.predict(X_test_imputed)
# mae = mean_absolute_error(y_test, predictions)

# results['KNNImputer'] = mae

# ## Multiple (Iterative) Imputation
# mice_imputer = IterativeImputer(random_state=42)
# X_train_imputed = mice_imputer.fit_transform(X_train)
# X_test_imputed = mice_imputer.transform(X_test)

# model.fit(X_train_imputed, y_train)
# predictions = model.predict(X_test_imputed)
# mae = mean_absolute_error(y_test, predictions)
# results['IterativeImputer'] = mae


# # Display results
# results_df = pd.DataFrame(list(results.items()), columns=['Imputation Method', 'MAE'])
# print(results_df)

# print(results_df.to_latex(index=False))


### Recategorization



In [ ]:
def categorize_rock(item):
  if item == 'MONZOGRANITE':
    return 'SYENITE'
  elif item in ['RHYOLITE', 'RHYODACITE']:
    return 'GRANITE'
  elif 'GRANITE' in item:
    return 'GRANITE'
  elif item in ['APLITE', 'PEGMATITE', 'GREISEN']:
    return 'GREISEN'
  elif item in ['DACITE', 'TONALITE', 'TRONDHJEMITE']:
    return 'GRANODIORITE'
  elif 'GRANODIORITE' in item:
    return 'GRANODIORITE'
  elif 'DIORITE' in item:
    return 'DIORITE'
  elif item == 'ANDESITE':
    return 'DIORITE'
  elif 'GABBRO' in item:
    return 'GABBRO'
  elif 'SYENITE' in item:
    return 'SYENITE'
  elif item in ['MONZONITE', 'MELASYENITE', 'PHONOLITE', 'TRACHYTE']:
    return 'SYENITE'
  elif 'CARBONATITE' in item:
    return 'CARBONATITE'
  elif item in ['LAMPROPHYRE', 'VOGESITE', 'NEPHELINITE']:
    return 'CARBONATITE'
  elif 'PHLOGOPITE' in item:
    return 'NOT GIVEN'
  elif 'XENOLITH' in item:
    return 'XENOLITH'
  elif item in ['CLINOPYROXENITE', 'HARZBURGITE', 'WEHRLITE', 'ECLOGITE']:
    return 'XENOLITH'
  else:
    return item

# clinopyroxenite, harzburgite, wehrlite, eclogite

In [ ]:
Cdf = df.copy()
Cdf['ROCK NAME'] = Cdf['ROCK NAME'].apply(categorize_rock)

In [ ]:
Cdf['ROCK NAME'].value_counts()

In [ ]:
# Shifting the latitude columns
Cdf['LATITUDE (MIN.)'] = Cdf['LATITUDE (MIN.)'] + 90
Cdf['LATITUDE (MAX.)'] = Cdf['LATITUDE (MAX.)'] + 90

# Shifting the longitude columns
Cdf['LONGITUDE (MIN.)'] = Cdf['LONGITUDE (MIN.)'] + 180
Cdf['LONGITUDE (MAX.)'] = Cdf['LONGITUDE (MAX.)'] + 180


In [ ]:
# Rectify negatives
def rectify_negatives(x):
    return x if x >= 0 else 1e-9

# Apply the function to the columns with negative values
columns_with_negatives = ['LI(PPM)', 'LA(PPM)',
                          'PR(PPM)', 'ND(PPM)']  # Replace with the actual column names
for col in columns_with_negatives:
    Cdf[col] = Cdf[col].apply(rectify_negatives)


In [ ]:
# Let us split the dataset first
arbitrary = ['LOCATION COMMENT',
              'LOCATION', 'SPOT', 'CRYSTAL',
             'RIM/CORE (MINERAL GRAINS)',
             'ALTERATION', 'ELEVATION (MIN.)','ELEVATION (MAX.)',
             'DRILLING DEPTH (MIN.)', 'DRILLING DEPTH (MAX.)',
             'ROCK TEXTURE']

Mdf = Cdf.drop(arbitrary, axis = 1)
Mdf.columns

# Data Imputation (KNN)

In [ ]:
# Create the KNN imputer instance
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors if needed

# Apply imputation on the numerical columns of the dataset
Mdf[numericals] = imputer.fit_transform(Mdf[numericals])

# One-Hot Encoding

In [ ]:
# One hot encoding
MCat = ['TECTONIC SETTING', 'ROCK NAME', 'PRIMARY/SECONDARY',
                'LAND/SEA (SAMPLING)', 'MINERAL']
# one-hot encode each column
for col in MCat:
  one_hot = pd.get_dummies(Mdf[col], prefix=col)
# Concatenate the one-hot encoded column back into the original df
  Mdf = pd.concat([Mdf, one_hot], axis = 1)
  Mdf = Mdf.drop(col, axis=1)

# Normalization



In [ ]:
# Splitting the dataset into train and test
Mdf_train, Mdf_test = train_test_split(Mdf, test_size=0.3, random_state=42)

# 1. Z-Score Normalization
scaler = StandardScaler()
Mdf_train_zscore = Mdf_train.copy()
Mdf_test_zscore = Mdf_test.copy()
Mdf_train_zscore[numericals] = scaler.fit_transform(Mdf_train[numericals])
Mdf_test_zscore[numericals] = scaler.transform(Mdf_test[numericals])

# 2. Log Transformation Normalization (Note: adding a small constant to avoid log(0))
Mdf_train_log = Mdf_train.copy()
Mdf_test_log = Mdf_test.copy()
Mdf_train_log[numericals] = Mdf_train[numericals].applymap(lambda x: np.log(x + 1e-9))
Mdf_test_log[numericals] = Mdf_test[numericals].applymap(lambda x: np.log(x + 1e-9))

# 3. Square Root Normalization
Mdf_train_sqrt = Mdf_train.copy()
Mdf_test_sqrt = Mdf_test.copy()
Mdf_train_sqrt[numericals] = np.sqrt(Mdf_train[numericals])
Mdf_test_sqrt[numericals] = np.sqrt(Mdf_test[numericals])



Train/Test Sets:

* Mdf_train_zscore, Mdf_test_zscore

* Mdf_train_log, Mdf_test_log

* Mdf_train_sqrt, Mdf_test_sqrt

####################################################################

# Models

# Random Forest

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define a function to run Random Forest and plot scatter
def run_rf_and_plot(X_train, y_train, X_test, y_test, title):
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Scatter plot of Predicted vs Actual
    plt.scatter(y_test, y_pred)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title(title + ' (Random Forest)')

    # Add line of best fit
    z = np.polyfit(y_test, y_pred, 1)
    p = np.poly1d(z)
    plt.plot(y_test, p(y_test), "r--", label='Line of Best Fit')

    # Save the plot
    plt.savefig(f"{title}.png")
    plt.show()

    return mse

# List of train-test sets
datasets = {
    "Z-Score": (Mdf_train_zscore, Mdf_test_zscore),
    "Log": (Mdf_train_log, Mdf_test_log),
    "Square Root": (Mdf_train_sqrt, Mdf_test_sqrt)
}

# Store results
results = {}

# Loop through each dataset and apply Random Forest
for name, (train, test) in datasets.items():
    X_train = train.drop("LI(PPM)", axis=1)
    y_train = train["LI(PPM)"]
    X_test = test.drop("LI(PPM)", axis=1)
    y_test = test["LI(PPM)"]

    mse = run_rf_and_plot(X_train, y_train, X_test, y_test, f"{name} Normalization MSE Scatter Plot")
    results[name] = mse


In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define a function to run Random Forest and plot scatter
def run_rf_and_plot(X_train, y_train, X_test, y_test, title):
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    mse = r2_score(y_test, y_pred)

    # Scatter plot of Predicted vs Actual
    plt.scatter(y_test, y_pred)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title(title + ' (Random Forest)')
    # Save the plot
    plt.savefig(f"{title}.png")
    plt.show()

    return mse

# List of train-test sets
datasets = {
    "Z-Score": (Mdf_train_zscore, Mdf_test_zscore),
    "Log": (Mdf_train_log, Mdf_test_log),
    "Square Root": (Mdf_train_sqrt, Mdf_test_sqrt)
}

# Store results
results = {}

# Loop through each dataset and apply Random Forest
for name, (train, test) in datasets.items():
    X_train = train.drop("LI(PPM)", axis=1)
    y_train = train["LI(PPM)"]
    X_test = test.drop("LI(PPM)", axis=1)
    y_test = test["LI(PPM)"]

    mse = run_rf_and_plot(X_train, y_train, X_test, y_test, f"{name} Normalization MSE Scatter Plot")
    results[name] = mse


In [ ]:
import pandas as pd

# Convert the results dictionary into a DataFrame
mse_df = pd.DataFrame(list(results.items()), columns=['Normalization Method', 'MSE'])

# Print the DataFrame for verification
print(mse_df)

# To convert DataFrame to LaTeX
latex_code = mse_df.to_latex(index=False)
print(latex_code)


In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error  # Importing mean_absolute_error instead of mean_squared_error

# # Define a function to run Random Forest
# def run_rf(X_train, y_train, X_test, y_test):
#     rf = RandomForestRegressor(n_estimators=100, random_state=42)
#     rf.fit(X_train, y_train)
#     y_pred = rf.predict(X_test)
#     return mean_absolute_error(y_test, y_pred)  # Changed this line to return MAE

# # List of train-test sets
# datasets = {
#     "Z-Score": (Mdf_train_zscore, Mdf_test_zscore),
#     "Log": (Mdf_train_log, Mdf_test_log),
#     "Square Root": (Mdf_train_sqrt, Mdf_test_sqrt)
# }

# # Store results
# results = {}

# # Loop through each dataset and apply Random Forest
# for name, (train, test) in datasets.items():
#     X_train = train.drop("LI(PPM)", axis=1)
#     y_train = train["LI(PPM)"]
#     X_test = test.drop("LI(PPM)", axis=1)
#     y_test = test["LI(PPM)"]

#     mae = run_rf(X_train, y_train, X_test, y_test)  # Storing MAE instead of MSE
#     results[name] = mae  # Storing MAE instead of MSE
#     print(f"{name} Normalization MAE: {mae}")  # Changed this line to print MAE


# Neural Network

In [ ]:


# Define a function to train a neural network
def run_nn(X_train, y_train, X_test, y_test, epochs=50):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation = 'relu', input_shape=(X_train.shape[1],)),
        Dropout(0.2),  # Add dropout with rate 0.2
        tf.keras.layers.Dense(64, activation = 'relu'),
        Dropout(0.2),  # Add dropout with rate 0.2
        tf.keras.layers.Dense(32, activation = 'relu'),
        Dropout(0.2),  # Add dropout with rate 0.2
        tf.keras.layers.Dense(16, activation = 'relu'),
        Dropout(0.2),  # Add dropout with rate 0.2
        tf.keras.layers.Dense(1, activation = 'linear')
    ])

    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['mse'])

    history = model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, verbose=0)
    eval_metrics = model.evaluate(X_test, y_test, verbose=0)
    return model, eval_metrics, history


In [ ]:
# Define a function to train a neural network
def run_nn(X_train, y_train, X_test, y_test, epochs=50):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.2),  # Add dropout with rate 0.2
        tf.keras.layers.Dense(64, activation='relu'),
        Dropout(0.2),  # Add dropout with rate 0.2
        tf.keras.layers.Dense(32, activation='relu'),
        Dropout(0.2),  # Add dropout with rate 0.2
        tf.keras.layers.Dense(16, activation='relu'),
        Dropout(0.2),  # Add dropout with rate 0.2
        tf.keras.layers.Dense(1, activation='linear')
    ])

    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['mse'])

    history = model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, verbose=0)

    # Get predictions on the test set
    y_pred = model.predict(X_test).flatten()

    # Calculate R^2 score
    r2 = r2_score(y_test, y_pred)

    eval_metrics = model.evaluate(X_test, y_test, verbose=0)

    return model, eval_metrics, history, r2

# Your code for running the NN models
nn_results = {}

for name, (train, test) in datasets.items():
    X_train = train.drop("LI(PPM)", axis=1)
    y_train = train["LI(PPM)"]
    X_test = test.drop("LI(PPM)", axis=1)
    y_test = test["LI(PPM)"]

    model, eval_metrics, history, r2 = run_nn(X_train, y_train, X_test, y_test)
    nn_results[name] = {'MSE': eval_metrics[0], 'R2': r2}  # Storing both MSE and R^2

# Convert the results dictionary into a DataFrame
nn_results_df = pd.DataFrame.from_dict(nn_results, orient='index').reset_index()
nn_results_df.columns = ['Normalization Method', 'MSE', 'R2']

# Print the DataFrame for verification
print(nn_results_df)

# To convert DataFrame to LaTeX
nn_latex_code = nn_results_df.to_latex(index=False)
print(nn_latex_code)


In [ ]:
# # List of train-test sets
# datasets = {
#     "Z-Score": (Mdf_train_zscore.drop("LI(PPM)", axis=1), Mdf_train_zscore["LI(PPM)"], Mdf_test_zscore.drop("LI(PPM)", axis=1), Mdf_test_zscore["LI(PPM)"]),
#     "Log": (Mdf_train_log.drop("LI(PPM)", axis=1), Mdf_train_log["LI(PPM)"], Mdf_test_log.drop("LI(PPM)", axis=1), Mdf_test_log["LI(PPM)"]),
#     "Square Root": (Mdf_train_sqrt.drop("LI(PPM)", axis=1), Mdf_train_sqrt["LI(PPM)"], Mdf_test_sqrt.drop("LI(PPM)", axis=1), Mdf_test_sqrt["LI(PPM)"])
# }

# # Store results
# nn_results = {}

# # Loop through each dataset and apply Neural Network
# for name, (X_train, y_train, X_test, y_test) in datasets.items():
#     model, eval_metrics, history = run_nn(X_train, y_train, X_test, y_test)
#     nn_results[name] = eval_metrics[1]  # Store the MSE
#     print(f"{name} Normalization Neural Network MSE: {eval_metrics[1]}")


In [ ]:
# Your code for running the NN models
nn_results = {}

for name, (train, test) in datasets.items():
    X_train = train.drop("LI(PPM)", axis=1)
    y_train = train["LI(PPM)"]
    X_test = test.drop("LI(PPM)", axis=1)
    y_test = test["LI(PPM)"]

    model, eval_metrics, history = run_nn(X_train, y_train, X_test, y_test)
    nn_results[name] = eval_metrics[0]  # Assuming that MSE is the first element in eval_metrics
import pandas as pd

# Convert the results dictionary into a DataFrame
nn_mse_df = pd.DataFrame(list(nn_results.items()), columns=['Normalization Method', 'MSE'])

# Print the DataFrame for verification
print(nn_mse_df)

# To convert DataFrame to LaTeX
nn_latex_code = nn_mse_df.to_latex(index=False)
print(nn_latex_code)

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

# Your existing datasets (replace these with your actual datasets)
datasets = {
    "Z-Score": (Mdf_train_zscore, Mdf_test_zscore),
    "Log": (Mdf_train_log, Mdf_test_log),
    "Square Root": (Mdf_train_sqrt, Mdf_test_sqrt)
}

# Initialize histories dictionary
histories = {}

# Run the model for each dataset and collect history
for name, (train, test) in datasets.items():
    X_train = train.drop("LI(PPM)", axis=1)
    y_train = train["LI(PPM)"]
    X_test = test.drop("LI(PPM)", axis=1)
    y_test = test["LI(PPM)"]

    # Running the neural network model
    _, _, history = run_nn(X_train, y_train, X_test, y_test)

    # Storing the history
    histories[name] = history

# Plotting the MSE curves
plt.figure(figsize=(10, 6))

for name, history in histories.items():
    plt.plot(history.history['mse'], label=f"{name} Train MSE")
    plt.plot(history.history['val_mse'], label=f"{name} Validation MSE")

plt.title('Neural Network MSE Curves')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.savefig('NNMSE.png')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

# Your existing datasets (replace these with your actual datasets)
datasets = {
    "Z-Score": (Mdf_train_zscore, Mdf_test_zscore),
    "Log": (Mdf_train_log, Mdf_test_log),
    "Square Root": (Mdf_train_sqrt, Mdf_test_sqrt)
}

# Initialize histories dictionary
histories = {}

# Mapping for custom legend names
legend_mapping = {
    "Z-Score": "M.1.1",
    "Log": "M.1.2",
    "Square Root": "M.1.3"
}

# Run the model for each dataset and collect history
for name, (train, test) in datasets.items():
    X_train = train.drop("LI(PPM)", axis=1)
    y_train = train["LI(PPM)"]
    X_test = test.drop("LI(PPM)", axis=1)
    y_test = test["LI(PPM)"]

    # Running the neural network model
    _, _, history = run_nn(X_train, y_train, X_test, y_test)

    # Storing the history
    histories[name] = history

# Plotting the MSE curves
plt.figure(figsize=(10, 6))

for name, history in histories.items():
    plt.plot(history.history['mse'], label=f"{legend_mapping[name]} Train MSE")
    plt.plot(history.history['val_mse'], label=f"{legend_mapping[name]} Validation MSE")

plt.title('Neural Network MSE Curves')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.savefig('NNMSE.png')
plt.show()


# Multivariate Polynomial Regression


In [ ]:
import matplotlib.pyplot as plt

# Function to run Polynomial Regression and plot
def run_polyreg_and_plot(X_train, y_train, X_test, y_test, degree=2, alpha=1.0, title=''):
    polyreg = make_pipeline(PolynomialFeatures(degree), Ridge(alpha=alpha))
    polyreg.fit(X_train, y_train)
    y_pred = polyreg.predict(X_test)
    mse = r2_score(y_test, y_pred)

    # Scatter plot of Predicted vs Actual
    plt.figure()
    plt.scatter(y_test, y_pred, color='purple')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title(title + ' (Polynomial Regression)')

    # Save the plot
    plt.savefig(f"{title}.png")
    plt.show()

    return mse

# Loop through each dataset and apply Polynomial Regression
polyreg_results = {}
for name, (train, test) in datasets.items():
    X_train = train.drop("LI(PPM)", axis=1)
    y_train = train["LI(PPM)"]
    X_test = test.drop("LI(PPM)", axis=1)
    y_test = test["LI(PPM)"]

    mse = run_polyreg_and_plot(X_train, y_train, X_test, y_test, title=f"{name} Normalization MSE Scatter Plot")
    polyreg_results[name] = mse
    print(f"{name} Normalization MSE: {mse}")


In [ ]:
import pandas as pd

# Create a DataFrame
mse_table = pd.DataFrame(list(polyreg_results.items()), columns=['Normalization', 'MSE'])

# To display the table in Python, just print it
print(mse_table)

# To export to LaTeX
print(mse_table.to_latex(index=False))


# XGBoost

In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Function to run XGBoost and plot scatter
def run_xgb_and_plot(X_train, y_train, X_test, y_test, title, n_estimators=100, learning_rate=0.1):
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=n_estimators, learning_rate=learning_rate)
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    mse = r2_score(y_test, y_pred)

    # Scatter plot of Predicted vs Actual
    plt.figure()
    plt.scatter(y_test, y_pred, color='red')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title(title)

    # Save the plot
    plt.savefig(f"{title}.png")

    # Show the plot
    plt.show()

    return mse

# List of train-test sets (replace these with your actual datasets)
datasets = {
    "Z-Score": (Mdf_train_zscore, Mdf_test_zscore),
    "Log": (Mdf_train_log, Mdf_test_log),
    "Square Root": (Mdf_train_sqrt, Mdf_test_sqrt)
}

# Store results
xgb_results = {}

# Loop through each dataset and apply XGBoost
for name, (train, test) in datasets.items():
    X_train = train.drop("LI(PPM)", axis=1)
    y_train = train["LI(PPM)"]
    X_test = test.drop("LI(PPM)", axis=1)
    y_test = test["LI(PPM)"]

    mse = run_xgb_and_plot(X_train, y_train, X_test, y_test, f"{name} Normalization XGBoost MSE Scatter Plot")
    xgb_results[name] = mse
    print(f"{name} Normalization XGBoost MSE: {mse}")


In [ ]:
# Generating LaTeX table for the MSE
latex_table = "\\begin{tabular}{|c|c|}\n\\hline\nNormalization & MSE \\\\\n\\hline"

for name, mse in xgb_results.items():
    latex_table += f"\n{name} & {mse:.4f} \\\\\n\\hline"

latex_table += "\\end{tabular}"
print(latex_table)


# Distributions before and after

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Features to examine
features_to_plot = ['AL2O3(WT%)', 'MGO(WT%)']

# Prepare datasets
all_datasets = {
    'Original': Mdf,
    'Z-Score': Mdf_train_zscore,  # Assuming you normalized the whole dataset
    'Log': Mdf_train_log,         # Assuming you normalized the whole dataset
    'Square Root': Mdf_train_sqrt # Assuming you normalized the whole dataset
}

# Loop through each feature
for feature in features_to_plot:
    plt.figure(figsize=(12, 8))

    # Loop through each dataset
    for name, dataset in all_datasets.items():
        sns.kdeplot(dataset[feature], label=f"{name}", shade=True)

    plt.title(f'Density Plot of {feature} After Different Normalizations')
    plt.xlabel(f'{feature}')
    plt.ylabel('Density')
    plt.legend()
    plt.savefig(f'BeforexAfter{feature}.png')
    plt.show()


############################################################
# Feature Engineering Tests

## Z Normalized

In [ ]:
# M2.1 Z Normalized, X imputation
M2_1_train = Mdf.sample(frac=0.8, random_state=42)
M2_1_val = Mdf.drop(M2_1_train.index)

# Handle missing values before normalization
M2_1imputer = SimpleImputer(strategy='mean')

# Perform imputation on training numerical features
M2_1imputed_data = M2_1imputer.fit_transform(M2_1_train[numericals])
M2_1_train[numericals] = pd.DataFrame(M2_1imputed_data, columns=numericals, index=M2_1_train.index)

# Perform imputation on validation numerical features with the same imputer
M2_1imputed_data = M2_1imputer.transform(M2_1_val[numericals])
M2_1_val[numericals] = pd.DataFrame(M2_1imputed_data, columns=numericals, index=M2_1_val.index)

# Normalize the numerical features using StandardScaler for Z-score normalization
M2_1scaler = StandardScaler()

M2_1_train[numericals] = M2_1scaler.fit_transform(M2_1_train[numericals])
M2_1_val[numericals] = M2_1scaler.transform(M2_1_val[numericals])

# Split the labels from the rest of the features
M2_1train_features = M2_1_train.copy()
M2_1val_features = M2_1_val.copy()

M2_1train_features = M2_1train_features.astype('float32')
M2_1val_features = M2_1val_features.astype('float32')

M2_1train_labels = M2_1train_features.pop('LI(PPM)')
M2_1val_labels = M2_1val_features.pop('LI(PPM)')


# MODEL 2

In [ ]:
# Normalized
model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu', input_shape=(M2_1train_features.shape[1],)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'linear')
])

model2.compile(loss='mean_squared_error', # MSE because we are predicting a continuous value
                   optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                      metrics = ['mse'])

history2_1 = model2.fit(M2_1train_features, M2_1train_labels, epochs=30,
                  batch_size = 32, validation_data = (M2_1val_features, M2_1val_labels))

In [ ]:
#  The above model evidences the drastic increase in model performance
# due to normalization. We will consider normalization of numeric features
# as a necessity. It will become part of our new baseline for future modifications.


# Next step is to examine data imputation techniques and whether we can
# maintain low loss and mse evaluations but with more 'fair'
# imputation techniques.

####################################################################
## FEATURE SELECTION & EXTRACTION
The model maintains exceedingly low loss and mse over its last few epochs.
The graph shows similar results, this could suggest either a perfect model or overfitting, with the latter being significantly more likely.

To combat overitting, we will perform feature selection/extraction. We wll examine the columns which benefited from the most data imputation.
the columns which had the most imputed values. These columns are likely where the discrepancy lies.
Less redundant data means less opportunity to make decisions based on noise in the data

For the sake of clarity, we will examine feature extraction on our so far most 'optimal' model, x.3 - the model with the lowest loss and mse evaluation metrics.
We will increment the number of features by 5 in each step, and we will key the optimal number feature as 'E'

In [ ]:
# EXAMINE FEATURES WITH 80% MISSING DATA
missing_value_threshold = 3763

# Identify columns which have missing values more than the threshold
columns_with_missing_values = [col for col in Mdf.columns if Mdf[col].isna().sum() >= missing_value_threshold]

print(len(columns_with_missing_values))


There are (obviously) no categorical features that need to be extracted - none benefit from data imputation.

Thus, we can just directly drop the relevant features from model 3.3

Labels also need not be touched, as they are LI(PPM)

In [ ]:
five = ['TE(PPM)', 'AU(PPM)', 'HG(PPM)', 'TL(PPM)', 'BI(PPM)']

ten = ['SE(PPM)', 'AG(PPM)', 'CD(PPM)', 'IN(PPM)', 'SB(PPM)',
       'TE(PPM)', 'AU(PPM)', 'HG(PPM)', 'TL(PPM)', 'BI(PPM)']

fifteen = ['DRILLING DEPTH (MAX.)','BE(PPM)', 'B(PPM)', 'GE(PPM)', 'AS(PPM)',
           'SE(PPM)', 'AG(PPM)', 'CD(PPM)', 'IN(PPM)', 'SB(PPM)',
           'TE(PPM)', 'AU(PPM)', 'HG(PPM)', 'TL(PPM)', 'BI(PPM)']

In [ ]:
# 5 features
M3_35train_features = M3_3train_features.drop(five, axis = 1)
M3_35val_features = M3_3val_features.drop(five, axis = 1)

In [ ]:
# 10 features
M3_310train_features = M3_3train_features.drop(ten, axis = 1)
M3_310val_features = M3_3val_features.drop(ten, axis = 1)

In [ ]:
# 15 features
M3_315train_features = M3_3train_features.drop(fifteen, axis = 1)
M3_315val_features = M3_3val_features.drop(fifteen, axis = 1)

In [ ]:
# # MODEL TESTING
# model35 = tf.keras.Sequential([
#     tf.keras.layers.Dense(128, activation = 'relu', input_shape=(M3_35train_features.shape[1],)),
#     tf.keras.layers.Dense(64, activation = 'relu'),
#     tf.keras.layers.Dense(32, activation = 'relu'),
#     tf.keras.layers.Dense(16, activation = 'relu'),
#     tf.keras.layers.Dense(1, activation = 'linear')
# ])

# model35.compile(loss='mean_squared_error', # MSE because we are predicting a continuous value
#                    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
#                       metrics = ['mse'])

# history3_35 = model35.fit(M3_35train_features, M3_3train_labels, epochs=30,
#                   batch_size = 32, validation_data = (M3_35val_features, M3_3val_labels))

In [ ]:
# # MODEL TESTING
# model310 = tf.keras.Sequential([
#     tf.keras.layers.Dense(128, activation = 'relu', input_shape=(M3_310train_features.shape[1],)),
#     tf.keras.layers.Dense(64, activation = 'relu'),
#     tf.keras.layers.Dense(32, activation = 'relu'),
#     tf.keras.layers.Dense(16, activation = 'relu'),
#     tf.keras.layers.Dense(1, activation = 'linear')
# ])

# model310.compile(loss='mean_squared_error', # MSE because we are predicting a continuous value
#                    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
#                       metrics = ['mse'])

# history3_310 = model310.fit(M3_310train_features, M3_3train_labels, epochs=30,
#                   batch_size = 32, validation_data = (M3_310val_features, M3_3val_labels))

In [ ]:
# MODEL TESTING
model315 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu', input_shape=(M3_315train_features.shape[1],)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'linear')
])

model315.compile(loss='mean_squared_error', # MSE because we are predicting a continuous value
                   optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                      metrics = ['mse'])

history3_315 = model315.fit(M3_315train_features, M3_3train_labels, epochs=30,
                  batch_size = 32, validation_data = (M3_315val_features, M3_3val_labels))

The models suggest that feature extraction has not rectified the overfitting on the data set.
The overfitting may be as a result of solely our imputation technique(s) or a combination of both imputation and noisy features.

As next steps, we will examine different imputation techniques, combine imputation with feature selection, and examine the effects of Regularization.
We will only consider the two normalized data sets.